In [29]:
from pytrends.request import TrendReq
import pandas as pd
import re
import matplotlib.pyplot as plt
import scipy.stats as sci
import numpy as np

df = pd.read_csv('symptoms.csv', sep=',',encoding='latin-1')


In [30]:
def word_in_bracket(x):
    match = re.search(r'.+?\(([A-Z].+)\)',x)
    if match is not None:
        return match.group(1)
    else:
        return x
    

In [33]:
df = pd.read_csv('symptoms.csv', sep=',',encoding='latin-1')
df.reset_index(drop=True)
df = df.drop(columns=['Unnamed: 0','[PathogenMitKrankheit].[Meldeweg Web71].[Meldeweg ID Web71].[MEMBER_CAPTION]'])
df = df.rename(columns={'[PathogenMitKrankheit].[Meldeweg Web71].[Krankheit ID Web71].[MEMBER_CAPTION]': 'Krankheit', '[Symptome].[ID].[ID].[MEMBER_CAPTION]': 'Symptom', '[ReportingDate].[YearWeek].[YearWeek].[MEMBER_CAPTION]': 'Kalenderwoche', '[Measures].[FallCount]': 'Anzahl'}) 


df_selection = df.drop(columns='Krankheit')
df_selection['Symptom'] = df_selection['Symptom'].apply(lambda x: 'Durchfall' if x =="Durchfall und/oder Erbrechen" else x)
df_selection['Symptom'] = df_selection['Symptom'].apply(lambda x: 'Durchfall' if x =="Durchfall, nicht näher bezeichnet" else x)
df_selection['Symptom'] = df_selection['Symptom'].apply(lambda x: 'Muskel-, Glieder-, Rücken- oder Kopfschmerzen' if x =="Gliederschmerzen" else x)
df_selection['Symptom'] = df_selection['Symptom'].apply(lambda x: 'Hirnhautentzündung' if x.startswith("Meningitis") else x)
df_selection['Symptom'] = df_selection['Symptom'].apply(word_in_bracket)


df_selection_pivot = df_selection.groupby(['Symptom','Kalenderwoche'])['Anzahl'].sum().reset_index()
df_selection_pivot.sort_values('Kalenderwoche')
df_selection_all_symptoms = df_selection.groupby(['Symptom'])['Anzahl'].sum().reset_index()['Symptom'].tolist()



new_rows = []
for week in df['Kalenderwoche'].drop_duplicates().sort_values():
    for symptom in df_selection_all_symptoms:
        sample_week = df_selection_pivot[df_selection_pivot['Kalenderwoche'] == week]
        if not sample_week.isin({'Symptom': [symptom]})['Symptom'].any():
            new_row = [symptom, week, 0]
            new_rows.append(new_row)
            
            
new_values = pd.DataFrame(new_rows, columns=['Symptom','Kalenderwoche','Anzahl'])

#append these illnesses with 0 'Anzahl'
df_selection_pivot= df_selection_pivot.append(new_values)
df_weekly_symptoms = df_selection_pivot.sort_values('Kalenderwoche').reset_index(drop=True)
df_weekly_symptoms = df_weekly_symptoms.sort_values(['Kalenderwoche','Symptom'],ascending=[True,True])





KeyboardInterrupt: 

In [237]:
#df_weekly_symptoms[df_weekly_symptoms['Symptom']=='Abszess']    
df_weekly_symptoms.to_pickle('df_weekly_symptoms.pkl')

In [35]:
df_dict_weeks_symptomps = {}
symptoms = set(df_weekly_symptoms["Symptom"])
symptoms = list(symptoms)
symptoms.sort()
dataframes = [df_weekly_symptoms[df_weekly_symptoms["Symptom"]=='{}'.format(symptom)] for symptom in symptoms]
dict_symptomps = dict(zip(symptoms,dataframes))

In [239]:
rki_values = np.zeros((len(dict_symptomps),261))
google_values = np.zeros((len(dict_symptomps),261))
pytrend = TrendReq()

for i, symptom in enumerate(dict_symptomps):
        rki_timeseries = dict_symptomps['{}'.format(symptom)]
        rki_timeseries = rki_timeseries[-261:]
        rki_values_unnormlised = rki_timeseries['Anzahl'].values
        rki_values[i,:] = rki_values_unnormlised/max(rki_values_unnormlised)+0.0000001
        
        try:
            pytrend.build_payload(kw_list=['{}'.format(symptom)],geo="DE",timeframe='today 5-y',cat=0)
            google_trend_per_syndrom = pytrend.interest_over_time()
            google_values_unnormlised = google_trend_per_syndrom['{}'.format(symptom)].values
            print(google_values_unnormlised)
            google_values[i,:] = google_values_unnormlised/max(google_values_unnormlised)+0.0000001
        except:
            print('exception')


exception
exception
exception
exception
exception


/Users/Auss/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in true_divide


exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception
exception


In [243]:
#pytrend = TrendReq()


test = 'Husten'
pytrend.build_payload(kw_list=['Husten'],geo="DE",timeframe='today 5-y',cat=0)
google_trend_per_syndrom = pytrend.interest_over_time()
google_values_unnormlised = google_trend_per_syndrom['Husten'].values
google_values= google_values_unnormlised/max(google_values_unnormlised)+0.0000001
print(google_values)


ResponseError: The request failed: Google returned a response with code 503.

In [226]:
correlation_values = np.zeros((len(dict_symptomps)))
for i in range(len(dict_symptomps)):
    try:
        output_tuple = sci.pearsonr(google_values[i],rki_values[i])
        correlation_values[i] = output_tuple[0]
    except RuntimeWarning:
        print('shits on fire')

/Users/Auss/anaconda/lib/python3.6/site-packages/scipy/stats/stats.py:3021: RuntimeWarning: invalid value encountered in double_scalars
  r = r_num / r_den


In [248]:
pytrend = TrendReq()
pytrend.build_payload(kw_list=["Abszess"],geo="DE",timeframe='today 5-y',cat=0)
over_time= pytrend.interest_over_time()
over_time = over_time.drop(columns='isPartial')

over_time['Abszess'].values
# over_time.plot(yticks=[0,100])
# plt.show()

ResponseError: The request failed: Google returned a response with code 503.